In [1]:
%load_ext autoreload

In [49]:
%autoreload

from allennlp.commands.elmo import ElmoEmbedder
import os
from sys import path
path.append('../..')
from relation_extraction.data import utils
import h5py
import numpy as np
dataset = 'i2b2' #TODO: update this
data_path = '/data/medg/misc/geeticka/relation_extraction/' + dataset + '/'
def res(path): return os.path.join(data_path, path)
out_path = 'pre-processed/original/elmo/'

In [50]:
if not os.path.exists(res(out_path)):
    os.makedirs(res(out_path))
if not os.path.exists(res(out_path + 'input-sentences')):
    os.makedirs(res(out_path + 'input-sentences'))

In [53]:
elmo = ElmoEmbedder()

## Getting all of the ELMo layers

In [59]:
border_sizes = [-1, 50, 1, 20]

In [60]:
def get_sentences(filename, border_size):
    data = open(res('pre-processed/original/' + filename))
    data = utils.split_data_cut_sentence(data, border_size)
    sentences = data[0]
    return sentences

In [61]:
def write_sentences_to_txt(sentences, filename):
    with open(res(out_path + 'input-sentences/'+filename + '.txt'), 'w') as f:
        for sentence in sentences:
            sentence_string = " ".join(sentence)
            f.write(sentence_string + "\n")

In [62]:
def write_hdf5(filename):
    with open(res(out_path + 'input-sentences/' + filename + '.txt')) as file:
        elmo.embed_file(input_file=file, 
                        output_file_path = res(out_path + filename + '.hdf5'))

In [ ]:
for border_size in border_sizes:
    train_sentences = get_sentences('train_original.txt', border_size)
    test_sentences = get_sentences('test_original.txt', border_size)
    write_sentences_to_txt(train_sentences, 'train_original_border_' + str(border_size))
    write_sentences_to_txt(test_sentences, 'test_original_border_' + str(border_size))
    write_hdf5('train_original_border_' + str(border_size))
    write_hdf5('test_original_border_' + str(border_size))

7833it [15:48,  8.26it/s]
14508it [27:16,  8.86it/s]
7833it [15:02,  8.68it/s]
14508it [29:00,  8.34it/s]
6941it [10:59, 10.53it/s]